In [1]:
from BatteryProject.ModelOne import trainer
from BatteryProject.ModelOne import model_params

In [2]:
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

In [3]:
features = model_params.features
features = features['feature_one']

In [10]:
model_1 = model_params.models['log'][0]
grid_1 = model_params.models['log'][1]

trainer_1 = trainer.Trainer()
trainer_1.features = features
trainer_1.get_data(trainer_1.features)
trainer_1.set_pipeline(model = model_1)
trainer_1.run(grid_1)

In [11]:
trainer_1.grid_search.best_params_

{'model__C': 0.2, 'model__penalty': 'l2', 'model__solver': 'lbfgs'}

In [12]:
trainer_1.eval()

{'accuracy': 0.9512195121951219,
 'precision': 0.9655172413793104,
 'roc_auc': 0.9410919540229885}

/home/romainj/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/romainj/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/romainj/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/romainj/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/romainj/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was

In [7]:
model_2 = model_params.models['random_forest'][0]
grid_2 = model_params.models['random_forest'][1]

trainer_2 = trainer.Trainer()
trainer_2.features = features
trainer_2.get_data(trainer_1.features)
trainer_2.set_pipeline(model = model_2)
trainer_2.run(grid_2)

/home/romainj/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
120 fits failed out of a total of 600.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
120 fits failed with the following error:
Traceback (most recent call last):
  File "/home/romainj/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/romainj/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/sklearn/pipeline.py", line 382, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/home/romainj/.pyenv/versions/3.8.12/envs/l

In [8]:
trainer_2.grid_search.best_params_

{'model__min_samples_leaf': 1,
 'model__min_samples_split': 2,
 'model__n_estimators': 50}

In [9]:
trainer_2.eval()

{'accuracy': 0.9024390243902439,
 'precision': 0.9583333333333334,
 'roc_auc': 0.908974358974359}